In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import pandas as pd


# Load the MovieLens dataset

In [30]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")
ratings=ratings.head(10000)
num_movies=int(ratings.max()[1])
num_users=int(ratings.max()[0])
print(num_movies,num_users)

190183 66


In [31]:
ratings = ratings.iloc[:,0:3]
ratings.min()



userId     1.0
movieId    1.0
rating     0.5
dtype: float64

In [112]:
# Y is the matrix containing all ratings
def load_dataset():
    Y= np.zeros((num_movies,num_users))
    R= np.zeros((num_movies,num_users))
    for row in ratings.iterrows():
        movieId= int(row[1][1])
        userId =int(row[1][0])
        rating=float(row[1][2])
        Y[movieId-1][userId-1]=rating
        R[movieId-1][userId-1]=int(1)
    return Y,R

    

## Normalize the dataset 

In [133]:
def normalize(Y,R):
    Ymean = (np.sum(Y*R,axis=1)/(np.sum(R, axis=1)+1e-12)).reshape(-1,1)
    Ynorm = Y - np.multiply(Ymean, R) 
    return Ymean,Ynorm


## Initialize variables to be learnt

In [196]:
num_features =20
# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users+1,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users+1),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

## Compute cost function

In [197]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):

    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [198]:

# Evaluate cost function 
J = cofi_cost_func_v(X, W, b, Y, R, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 3045634.21


## Minimize the cost function

In [199]:

iterations = 250
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
#     if iter % 20 == 0:
    print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 2024653.4
Training loss at iteration 1: 1694627.3
Training loss at iteration 2: 1419089.4
Training loss at iteration 3: 1188754.8
Training loss at iteration 4: 995543.6
Training loss at iteration 5: 833079.9
Training loss at iteration 6: 696669.3
Training loss at iteration 7: 582530.1
Training loss at iteration 8: 487348.5
Training loss at iteration 9: 408173.7
Training loss at iteration 10: 342521.4
Training loss at iteration 11: 288335.5
Training loss at iteration 12: 243834.3
Training loss at iteration 13: 207406.1
Training loss at iteration 14: 177616.3
Training loss at iteration 15: 153258.2
Training loss at iteration 16: 133345.3
Training loss at iteration 17: 117050.3
Training loss at iteration 18: 103657.3
Training loss at iteration 19: 92559.5
Training loss at iteration 20: 83271.1
Training loss at iteration 21: 75414.4
Training loss at iteration 22: 68690.2
Training loss at iteration 23: 62855.5
Training loss at iteration 24: 57720.3
Training los

Training loss at iteration 215: 497.5
Training loss at iteration 216: 497.2
Training loss at iteration 217: 497.0
Training loss at iteration 218: 496.8
Training loss at iteration 219: 496.6
Training loss at iteration 220: 496.4
Training loss at iteration 221: 496.2
Training loss at iteration 222: 496.0
Training loss at iteration 223: 495.8
Training loss at iteration 224: 495.6
Training loss at iteration 225: 495.4
Training loss at iteration 226: 495.2
Training loss at iteration 227: 495.0
Training loss at iteration 228: 494.8
Training loss at iteration 229: 494.6
Training loss at iteration 230: 494.4
Training loss at iteration 231: 494.3
Training loss at iteration 232: 494.1
Training loss at iteration 233: 493.9
Training loss at iteration 234: 493.7
Training loss at iteration 235: 493.6
Training loss at iteration 236: 493.4
Training loss at iteration 237: 493.2
Training loss at iteration 238: 493.1
Training loss at iteration 239: 492.9
Training loss at iteration 240: 492.7
Training los

In [192]:
my_ratings = np.zeros(num_movies)          #  Initialize my ratings

# Check the file movies.csv for id of each movie in our dataset
# For example, Zone 39(1997) has ID 2698, so to rate it "5", you can set
my_ratings[2698-1] = 5 

#Or suppose you did not enjoy Winslow Boy, The (1999), you can set
my_ratings[2361-1] = 2;

# Lets try rating action and war movies highly to see if movie works

my_ratings[5995-1]=4.90; # Pianist
my_ratings[527-1]=5; #Schindlers List
my_ratings[2427-1]=4.55; #Thin Red Line, The (1998)
my_ratings[1927-1]=4.62; #All Quiet on the Western Front
my_ratings[2028-1]=4.51; #Saving Private Ryan
my_ratings[912-1]=4.1; # Casablanca
my_ratings[1208-1]=4.0 #Apocalypse now
my_ratings[110-1]=4.2 #Braveheart
my_ratings[1222-1]=4.74 #Full Metal Jacket

my_ratings[1721-1]=3.5; # Titanic
my_ratings[1704-1]=3; #Good Will Hunting
my_ratings[356-1]=2.5; # Forrest Gump
my_ratings[1202-1]=2; #Withnail
my_ratings[39-1]=3.1 #Clueless
my_ratings[276-1]=3.8 #Milk money

# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
 
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]
print(my_ratings[355])


2.5


In [175]:
movieList_df= pd.read_csv("ml-latest-small/movies.csv")
movieList_df=movieList_df.loc[:,['movieId','title']]

In [193]:
print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        movie=movieList_df.loc[movieList_df["movieId"]==i+1].values[0][1]
        print(f'Rated {my_ratings[i]} for  {movie}');


New user ratings:

Rated 3.1 for  Clueless (1995)
Rated 4.2 for  Braveheart (1995)
Rated 3.8 for  Milk Money (1994)
Rated 2.5 for  Forrest Gump (1994)
Rated 5.0 for  Schindler's List (1993)
Rated 4.1 for  Casablanca (1942)
Rated 2.0 for  Withnail & I (1987)
Rated 4.0 for  Apocalypse Now (1979)
Rated 4.74 for  Full Metal Jacket (1987)
Rated 3.0 for  Good Will Hunting (1997)
Rated 3.5 for  Titanic (1997)
Rated 4.62 for  All Quiet on the Western Front (1930)
Rated 4.51 for  Saving Private Ryan (1998)
Rated 2.0 for  Pink Flamingos (1972)
Rated 4.55 for  Thin Red Line, The (1998)
Rated 5.0 for  Zone 39 (1997)
Rated 4.9 for  Pianist, The (2002)


## Now we reload dataset and add these rating to matrix Y

In [194]:
# Reload ratings
Y, R = load_dataset()


# Add new user ratings to Y 
Y = np.c_[my_ratings, Y]
print(Y.shape)

# Add new user indicator matrix to R
R = np.c_[(my_ratings != 0).astype(int), R]


# Normalize the Dataset
Ymean,Ynorm= normalize(Y, R)

(190183, 67)


In [164]:
movieList=['' for _ in range(193609+1)]
print(num_movies)
for row in movieList_df.iterrows():
    id=int(row[1][0])
    mname=row[1][1]
    print(id)
    movieList[id]=mname
print(movieList)

190183
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
34
36
38
39
40
41
42
43
44
45
46
47
48
49
50
52
53
54
55
57
58
60
61
62
63
64
65
66
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
85
86
87
88
89
92
93
94
95
96
97
99
100
101
102
103
104
105
106
107
108
110
111
112
113
116
117
118
119
121
122
123
125
126
128
129
132
135
137
140
141
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
168
169
170
171
172
173
174
175
176
177
178
179
180
181
183
184
185
186
187
188
189
190
191
193
194
195
196
198
199
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
222
223
224
225
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
265
266
267
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
287
288
289
290
291
292
293
294
295
296
298
299
300
301
302
303
304
305
306
307
308
310
311
312
31

27788
27790
27793
27801
27802
27803
27808
27815
27816
27820
27821
27822
27826
27829
27830
27831
27834
27837
27838
27839
27846
27850
27865
27866
27869
27873
27878
27879
27882
27899
27904
27905
27912
30707
30745
30749
30793
30803
30810
30812
30816
30818
30820
30822
30825
30846
30848
30850
30883
30890
30892
30894
30898
30994
31000
31030
31038
31049
31083
31086
31101
31114
31116
31123
31150
31162
31184
31193
31221
31223
31225
31260
31297
31309
31364
31367
31410
31420
31422
31424
31427
31431
31433
31435
31437
31445
31522
31545
31553
31590
31610
31617
31658
31660
31664
31685
31692
31694
31696
31698
31700
31702
31737
31804
31851
31867
31878
31903
31909
31921
31923
31925
31952
31973
32009
32011
32017
32019
32022
32029
32031
32058
32060
32116
32139
32160
32179
32213
32234
32243
32289
32291
32294
32296
32298
32300
32302
32314
32371
32387
32392
32440
32442
32456
32460
32469
32511
32515
32554
32582
32584
32587
32589
32596
32598
32600
32620
32632
32649
32657
32659
32666
32728
32743
32770
32799
3286

## Train the model by running previous cells again

### Let's make recommendations now

In [200]:
# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
pm = p + Ymean

my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j+1 not in my_rated:
        movie=movieList_df.loc[movieList_df["movieId"]==j+1].values[0][1]
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movie}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        movie=movieList_df.loc[movieList_df["movieId"]==i+1].values[0][1]
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movie}')

Predicting rating 4.97 for movie Zone 39 (1997)
Predicting rating 4.96 for movie The Raid 2: Berandal (2014)
Predicting rating 4.96 for movie Hannibal Rising (2007)
Predicting rating 4.96 for movie Black Mirror: White Christmas (2014)
Predicting rating 4.96 for movie Predestination (2014)
Predicting rating 4.96 for movie The Godfather Trilogy: 1972-1990 (1992)
Predicting rating 4.96 for movie Planet Terror (2007)
Predicting rating 4.96 for movie Black Mirror
Predicting rating 4.96 for movie Jiro Dreams of Sushi (2011)
Predicting rating 4.96 for movie Shoot 'Em Up (2007)
Predicting rating 4.96 for movie The Man from Nowhere (2010)
Predicting rating 4.95 for movie Education, An (2009)
Predicting rating 4.95 for movie First Daughter (2004)
Predicting rating 4.95 for movie Idiocracy (2006)
Predicting rating 4.95 for movie Romeo and Juliet (1968)
Predicting rating 4.94 for movie Home for the Holidays (1995)
Predicting rating 4.94 for movie Adventures of Pinocchio, The (1996)


Original vs P